In [1]:
import glob, os
import numpy as np
import geopandas as gpd
import rasterio
import rasterio.mask
import rasterio.plot
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject, Resampling
import matplotlib.pyplot as plt
from pathlib import Path

print('All libraries successfully imported!')
print(f'Rasterio : {rasterio.__version__}')

All libraries successfully imported!
Rasterio : 1.2.1


In [2]:
im_file = '/export/homes/students/dominiquem/Occup_sol/Wal_OCS_2018_clipLiege.tif'

im_file_reproj = '/export/homes/ndeffense/SHARE/Wal_OCS_2018_clipLiege_reproj.tif'

In [5]:

# Open file
src = rasterio.open(im_file, "r")

crs = '3812'
crs_tif = str(src.crs).split(":",1)[1]

print(crs_tif)

    
crs_dst = {'init': f'EPSG:{crs}'}

transform, width, height = calculate_default_transform(src.crs,
                                                        crs_dst,
                                                        src.width,
                                                        src.height,
                                                        *src.bounds)
                                                    

# Update metadata
out_meta = src.meta.copy()

out_meta.update(driver = 'GTiff',
                crs = crs_dst,
                width = width,
                height = height,
                transform = transform)

print(out_meta)

'''
#open destination raster
dst = rasterio.open(im_file_reproj, 'w', **out_meta)

#reproject and save raster band data
for i in range(1, src.count + 1):
    reproject(
        source = rasterio.band(src, i),
        destination = rasterio.band(dst, i),
        #src_transform=src.transform,
        src_crs = src.crs,
        #dst_transform=transform,
        dst_crs = crs_dst,
        resampling = Resampling.nearest)

# Close rasterio objects
src.close()
dst.close()

print(f'--> A new reprojected raster file is created : {im_file_reproj}')
'''

31370
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 101937, 'height': 75308, 'count': 1, 'crs': {'init': 'EPSG:3812'}, 'transform': Affine(1.0000028313320035, 0.0, 693247.6090646925,
       0.0, -1.0000028313320035, 667737.724611248)}


"\n#open destination raster\ndst = rasterio.open(im_file_reproj, 'w', **out_meta)\n\n#reproject and save raster band data\nfor i in range(1, src.count + 1):\n    reproject(\n        source = rasterio.band(src, i),\n        destination = rasterio.band(dst, i),\n        #src_transform=src.transform,\n        src_crs = src.crs,\n        #dst_transform=transform,\n        dst_crs = crs_dst,\n        resampling = Resampling.nearest)\n\n# Close rasterio objects\nsrc.close()\ndst.close()\n\nprint(f'--> A new reprojected raster file is created : {im_file_reproj}')\n"

In [13]:
import subprocess

crs = '3812'

cmd = 'gdalwarp'
cmd += ' -of Gtiff'
cmd += ' -co BIGTIFF=YES -co COMPRESS=LZW -co TILED=YES -co NUM_THREADS=ALL_CPUS'
cmd += f' -t_srs EPSG:{crs}'
cmd += ' ' + im_file
cmd += ' ' + im_file_reproj

print(cmd)

subprocess.call(cmd, shell=True)

gdalwarp -of Gtiff -co BIGTIFF=YES -co COMPRESS=LZW -co TILED=YES -co NUM_THREADS=ALL_CPUS -t_srs EPSG:3812 /export/homes/students/dominiquem/Occup_sol/Wal_OCS_2018_clipLiege.tif /export/homes/ndeffense/SHARE/Wal_OCS_2018_clipLiege_reproj.tif


1

In [33]:
im_file = '/export/miro/students/dominiquem/WORK/PSF/occu_constru.tif'
im_file_out = '/export/miro/ndeffense/SHARE/occu_constru_downscaling_average_5x5_tap.tif'

cmd = 'gdalwarp'
cmd += ' -tr 5 5'
cmd += ' -tap'
cmd += f' -s_srs EPSG:31370'
#cmd += f' -t_srs EPSG:3812'
cmd += ' -r average '
cmd += ' -ot Float32'
cmd += f' {im_file}'
cmd += f' {im_file_out}'

print(cmd)

subprocess.call(cmd, shell=True)

gdalwarp -tr 5 5 -tap -s_srs EPSG:31370 -r average  -ot Float32 /export/miro/students/dominiquem/WORK/PSF/occu_constru.tif /export/miro/ndeffense/SHARE/occu_constru_downscaling_average_5x5_tap.tif


1

In [29]:
im_file     = '/export/miro/students/dominiquem/WORK/PSF/occu_constru.tif'
im_file_out = '/export/miro/ndeffense/SHARE/occu_constru_downscaling_sum_numpy.tif'

src = rasterio.open(im_file, "r")
img_in = src.read(1)

print(img_in)

print(img_in.shape)

img_in = img_in[:7530, :9060]

print(img_in.shape)

scale = 30

#orig_height = src.height
#orig_width  = src.width

orig_height = 7530
orig_width  = 9060

target_width  = int(orig_width / scale)
target_height = int(orig_height / scale)

print(target_width)
print(target_height)


img_in_coarse = np.nansum(img_in.reshape(-1, scale), 1).reshape(orig_height, -1)
img_in_coarse = np.nansum(img_in_coarse.reshape(-1, scale, target_width), 1).reshape(target_height, -1)

img_in_coarse = img_in_coarse.astype('int16')

print(img_in_coarse)

print(img_in_coarse.shape)


print(src.transform)

new_transform = src.transform * src.transform.scale(
    (orig_height / img_in_coarse.shape[-1]),
    (orig_width / img_in_coarse.shape[-2])
)

print(new_transform)

profile = src.profile
profile.update(driver = 'GTiff',
                width = target_width,
                height = target_height,
                transform = new_transform)

print(profile)

# Write resampled image
dst = rasterio.open(im_file_out, "w", **profile)
dst.write(img_in_coarse,1)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(7532, 9062)
(7530, 9060)
302
251
[[186 331 124 ... 424   0   0]
 [425 348 162 ... 134  33   0]
 [326 120  35 ...   0 107 213]
 ...
 [509  15   0 ...   0   0   0]
 [489 128 304 ...   0   0   0]
 [418 261 501 ...   0   0   0]]
(251, 302)
| 1.00, 0.00, 231736.38|
| 0.00,-1.00, 151412.78|
| 0.00, 0.00, 1.00|
| 24.93, 0.00, 231736.38|
| 0.00,-36.10, 151412.78|
| 0.00, 0.00, 1.00|
{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -10000.0, 'width': 302, 'height': 251, 'count': 1, 'crs': CRS.from_wkt('PROJCS["Belge_Lambert_1972",GEOGCS["Belge 1972",DATUM["Reseau_National_Belge_1972",SPHEROID["International 1924",6378388,297.000000000005,AUTHORITY["EPSG","7022"]],AUTHORITY["EPSG","6313"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4313"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",90],PARAMET